## `CNLS` with multiplicative error

  - Author: Sheng Dai (sheng.dai@aalto.fi)
  - Date   : June 13, 2020
  
References:

[1] Kuosmanen, T., Johnson, A. and Saastamoinen, A. (2015). Stochastic Nonparametric Approach to Efficiency Analysis: A unified Framework, in Zhu, J. (ed.) Data Envelopment Analysis. Springer, pp. 191–244.  

### Example

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the package pystoned
from pystoned import CNLS

In [3]:
# import OECD countries data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/countries.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head(5)

,Country,Year,HRSN,CPNK,VALK,GHG
0,AUS,2010,19614.5,2952176.5,914377.0,537.2
1,AUS,2011,19949.9,3071779.7,951464.7,538.5
2,AUS,2012,20060.5,3192681.1,977274.9,541.3
3,AUS,2013,20181.6,3300697.4,1003749.5,531.3
4,AUS,2014,20359.3,3391029.7,1027981.2,525.8


In [4]:
# output
y = df['VALK']

# inputs
x1 = df['HRSN']
x1 = np.asmatrix(x1).T
x2 = df['CPNK']
x2 = np.asmatrix(x2).T
x3 = df['GHG']
x3 = np.asmatrix(x3).T
x = np.concatenate((x1, x2, x3), axis=1)

In [5]:
# define and solve the CNLS model

cet = "mult"
fun = "prod"
rts = "crs"

model = CNLS.cnls(y, x, cet, fun, rts)

# using remote solver (NEOS)
from pyomo.environ import SolverManagerFactory
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

In [6]:
# display the estimates (beta and residual)
model.b.display()
model.e.display()

b : beta
    Size=504, Index=b_index
    Key      : Lower : Value                  : Upper : Fixed : Stale : Domain
      (0, 0) :   0.0 :   0.019550714474662084 :  None : False : False :  Reals
      (0, 1) :   0.0 :    0.23854425509029353 :  None : False : False :  Reals
      (0, 2) :   0.0 :     357.47244477248046 :  None : False : False :  Reals
      (1, 0) :   0.0 :    0.01931274212460568 :  None : False : False :  Reals
      (1, 1) :   0.0 :    0.23854444508345715 :  None : False : False :  Reals
      (1, 2) :   0.0 :      357.4801141185179 :  None : False : False :  Reals
      (2, 0) :   0.0 :     16.534790658439086 :  None : False : False :  Reals
      (2, 1) :   0.0 :     0.1950161577835109 :  None : False : False :  Reals
      (2, 2) :   0.0 :      2.152607881486465 :  None : False : False :  Reals
      (3, 0) :   0.0 :    0.19457473567490702 :  None : False : False :  Reals
      (3, 1) :   0.0 :    0.23803964833053864 :  None : False : False :  Reals
      (3, 2) : 

In [7]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)
eps

array([ 0.01958582,  0.02751254,  0.02254811,  0.02631336,  0.03033455,
        0.03713118, -0.15005356, -0.1273061 , -0.13071926, -0.14127659,
       -0.13883609, -0.14287756,  0.06993553,  0.0944756 ,  0.09125628,
        0.08538886,  0.09448389,  0.09490995,  0.29605879,  0.2998097 ,
        0.28709217,  0.284416  ,  0.29133541,  0.28635482, -0.40603386,
       -0.3958171 , -0.41157828, -0.42108231, -0.39334017, -0.36057503,
        0.01286188,  0.04754454,  0.06552052,  0.06909295,  0.08583538,
        0.09624383, -0.27232764, -0.25514409, -0.24175771, -0.24713589,
       -0.24793907, -0.27045247, -0.01239089,  0.01437485, -0.00163494,
       -0.01639373, -0.02071386, -0.01755803,  0.03105757,  0.06020249,
        0.06187143,  0.05908223,  0.06882998,  0.0641569 , -0.01135406,
        0.02016695,  0.01922371,  0.01742997,  0.03631429,  0.04589166,
       -0.03550182, -0.11882895, -0.16426966, -0.17614657, -0.1541725 ,
       -0.24475822, -0.38250169, -0.36210109, -0.36786601, -0.33

In [8]:
# retrive the beta
ind = list(model.b)
val = list(model.b[:, :].value)
beta = np.asarray([i + tuple([j]) for i, j in zip(ind, val)])

beta = pd.DataFrame(beta, columns=['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2', 'b3']
beta

,b1,b2,b3
Name,,,
0.0,0.019551,0.238544,357.472445
1.0,0.019313,0.238544,357.480114
2.0,16.534791,0.195016,2.152608
3.0,0.194575,0.238040,353.958718
4.0,0.028647,0.238504,357.377608
...,...,...,...
163.0,0.007594,0.238586,357.682691
164.0,0.001831,0.238588,357.878715
165.0,0.001860,0.238588,357.876235
